# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Dataset
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import joblib
import os
from azureml.data.dataset_factory import TabularDatasetFactory

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive_experiment'

experiment=Experiment(ws, experiment_name)

In [ ]:
#Source of the data: https://raw.githubusercontent.com/aiplanethub/Datasets/master/HR_comma_sep.csv

# We first check if the data is already loaded inthe Workspace. Otherwise, create it from the file

found = False
key = "HR Dataset"
description_text = "HR Dataset for capstone project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/aiplanethub/Datasets/master/HR_comma_sep.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

## Cluster creation

In [ ]:
cluster_name = "cluster-udacity"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster in the provided workspace, so we use it.')
except ComputeTargetException:
    comp_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=3)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, comp_config)

cpu_cluster.wait_for_completion(show_output=True)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
import numpy as np

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, 
                                        evaluation_interval = 2,
                                        delay_evaluation = 10)

#TODO: Create the different params that you will be using during training
max_depth_range = np.arange(4, 10)
min_samples_leaf_range = np.arange(2,10)

param_sampling = RandomParameterSampling( {
        '--criterion': ['entropy', 'gini'],
        '--max_depth': max_depth_range,
        '--min_samples_leaf': min_samples_leaf_range
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')


#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory='.',
                            command=['python', 'train.py'],
                            compute_target=cluster_name,
                            environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator, 
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name='AUC_weighted',
                             primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs = 4,
                             max_concurrent_runs = 2)

In [ ]:
#TODO: Submit your experiment

# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = experiment.submit(config=hyperdrive_run_config) 



## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
#show run details with the widget.
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
if best_run is None:
    raise Exception("No best run was found")

print(best_run)

best_run_params = best_run.get_details()['runDefinition']['parameters']
best_run_metrics = best_run.get_metrics()

print("Information about the best model: \n")
print("Run id: {}".format(best_run.id))
print("Accuracy: {}".format(best_run_metrics['Accuracy']))
print("Hyperparameter values: \n")
print(dict({best_run_params[1::2][0]['name'][9:]: best_run_params[1::2][0]['value']}))
print(dict({best_run_params[1::2][1]['name'][9:]: best_run_params[1::2][1]['value']}))

model_name = "HyperDrive_model.pkl"
file_name = "outputs/" + model_name

#if not os.path.exists('outputs/'):
#    os.makedirs('outputs/')
#joblib.dump(value=best_run, filename=file_name)


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#Register the model
description = 'HyperDrive best forescasting model'
model = best_run.register_model(model_name='HyperDriveModel', description=description, model_path=file_name)
print(model)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

